# Urban canyon study

This example shows the behavior of sparrowpy in the simulation of an urban canyon under different simulation conditions

In [ ]:
"""Test the radiosity.Radiosity module."""
import pyfar as pf
import sparrowpy as sp
from run_simu import run_simu, run_simu_pure
import numpy as np
import matplotlib.pyplot as plt
import os

%load_ext memory_profiler

%matplotlib inline
# %matplotlib ipympl

basepath=os.path.join(os.getcwd(),"out")

Lets define our room and source position.

In [ ]:
def urban_canyon_stub(length_x, length_y, length_z):
    """Create a shoebox room with the given dimensions.

    Parameters
    ----------
    length_x : float
        Length of the urban canyon in meters.
    length_y : float
        width of the urban canyon in meters.
    length_z : float
        height of the urban canyon in meters.

    Returns
    -------
    room : list[geo.Polygon]
        List of the walls of the room.

    """
    return [
        sp.geometry.Polygon(
            [[0, 0, 0], [length_x, 0, 0],
            [length_x, 0, length_z], [0, 0, length_z]],
            [0, 0, 1], [0, 1, 0]),
        sp.geometry.Polygon(
            [[0, 0, 0],[0, length_y, 0],
            [length_x, length_y, 0], [length_x, 0, 0 ]],
            [1, 0, 0], [0, 0, 1]),
        sp.geometry.Polygon(
            [[0, length_y, 0], [length_x, length_y, 0],
            [length_x, length_y, length_z], [0, length_y, length_z]],
            [0, 0, -1], [0, -1, 0]),
        ]

In [ ]:
# Define parameters
X = 90
Y = 12
Z = 6

# create geometry
canyon = urban_canyon_stub(X, Y, Z)
source = pf.Coordinates(20, 6, 1)
receiver = pf.Coordinates(70, 6, 2)

In [ ]:
run_simu(walls=urban_canyon_stub(1,1,1),
        source=pf.Coordinates(.5,.5,.5),
        receiver=pf.Coordinates(.5,.5,.5),
        patch_size=1)

## diffuse vs custom

In [ ]:
f_array = np.array([250, 500, 1000, 2000, 4000])
airatt = np.array([1.309749765923859,2.7281341349384447, 4.664731873821475,
                   9.88701558869792,29.665528426392616])/1000
order=30
dur=1.
step=.002
psize=2

In [ ]:
filename = "resources\\triangle_sim_optimal.s_d.sofa"
etc_diff=run_simu_pure( walls=canyon,
                        source=source,
                        receiver=receiver,
                        patch_size=psize,
                        time_step=step,
                        duration=dur,
                        refl_order=order,
                        freq=f_array,
                        att=airatt,
                        file=filename,
                        classical_model=True)


In [ ]:

etc_file=run_simu_pure( walls=canyon,
                        source=source,
                        receiver=receiver,
                        patch_size=psize,
                        time_step=step,
                        duration=dur,
                        refl_order=order,
                        freq=f_array,
                        att=airatt,
                        file=filename,
                        classical_model=False)

In [ ]:
pf.io.write(os.path.join(basepath,"proof_etcs.far"),compress=False,
            diffuse_etc=etc_diff,
            custom_etc=etc_file,
            freqs=f_array)

In [ ]:
pf.plot.time(etc_file, dB=True, log_prefix=10,
             label="custom scattering",
             linestyle="-")
pf.plot.time(etc_diff, dB=True, log_prefix=10,
             label="diffuse",
             linestyle="--")
plt.legend()

In [ ]:
%load_ext watermark
%watermark -v -m -iv